Urban Data Science & Smart Cities <br>
URSP688Y <br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

[<img src="https://colab.research.google.com/assets/colab-badge.svg">](https://colab.research.google.com/github/ncsg/ursp688y_sp2024/blob/main/exercises/exercise04/exercise04.ipynb)

# Exercise 4 (in Two Parts)

# The Data Viz Part

Next week is Data Visualization week. This is one of my favorite topics, in part because we get to look at lots of pictures, and in part because it provides an excuse for some very lighthearted competition.

In prep for next week, part of your exercise is to find an example of either an _excellent_ or _terrible_ data visualization. We will vote on the best (and worst) in each category, and the winner gets a small (tasty) prize.

Please find an example of a data visualization that is either _very effective_ or _terribly ineffective_ in communicating an interesting finding from data. Here are a few ground rules:
- One figure only: We should be able to see the whole thing at once on a projector screen.
- Static images only: If you find something dynamic or interactive that you _must_ submit, please take a screenshot.
- Do the reading first: Tufte will give you some ideas for what makes visualizations good or bad
- No examples from Tufte. Gotta work a little bit.

Please either paste a link to your image in the text cell below (can you figure out how to get markdown to display the image?) or add an image file to your PR.
- Please label it clearly as "good" or "bad" so we know which race you're in.
- Please write a couple bullets about why it's good or bad. This is your pitch (we can haggle about it in class, too.)

## Good/Bad (please choose one and delete the other)
- Why
- Some more why
- Any more?

***** Put image link or insert image here *****

# The Programming Part

## Problem

In [Exercise 3](https://github.com/ncsg/ursp688y_sp2024/blob/main/exercises/exercise03/exercise03.ipynb), you examined how many affordable housing units available to households up to 60% AMI were planned within each ward in Washington, D.C.

The bonus problem was to calculate which wards were producing a _disproportionately_ large and small number of housing units given their populations.

This week, please reproduce this analysis, <ins>including</ins> the bonus part, using some of your new data loading, joining, and module-building skills.

Please write a program that:

- Loads the affordable housing project data from `affordable_housing.csv`
- Loads the ward populations from `wards_from_2022.csv`
- Joins the population data to the affordable housing data
- Calculates which wards are producing disproportionately large and small number of housing units given their populations
- Completes all of this data loading and processing within a function (or a series of functions called by a single main function)
- Stores that function (and any related functions) in a module
- Calls the main function in the exercise notebook to return table or other summary or results

## Data

CSVs for both required data tables are included on GitHub at `exercises/exercise04`.

Please consult the city's database of [affordable housing](https://opendata.dc.gov/datasets/DCGIS::affordable-housing/about) projects and [ward demographic](https://opendata.dc.gov/datasets/DCGIS::wards-from-2022/about) data.

Bonus: find, download, and use more recent ward population data. (Remember to include it in your PR.) My cursory search found data as late as 2022.

## New instructions for submitting a PR with multiple files

Because you'll be working with multiple files, PRs become _slightly_ more complicated, so we're graduating to a new 'mini-repository' pattern:
- Make a new folder in `exercises/exercise04` with your last name (just like the suffix for your notebook file)
- Upload your notebook file, also appropriately named, into that folder
- Upload any other files you make/use, including `.py` and `.csv` files, into that folder, so everything is together in the same place

Ultimately, this will look a bit like this:
```
── exercises
    ├── exercise04
        ├── harvey
            ├── exercise04_harvey.ipynb
            ├── affordable_housing_calcs.py
            ├── affordable_housing.csv
            └── wards_from_2022.csv
```

**NOTE:** Yes, I realize this is a bit redundant because everyone will have copies of the same CSV files. This would never be a good idea for production coding--we would have one `data` directory, and everyone would draw from the same data. However, there are two reasons for all these copies in this case:
1. It's good practice to build a repository with all the parts your code needs to run.
    - In later weeks, when you  _don't_ all have the same data, it won't seem as redundant.
3. Having everything in one folder will make it easy for me to run your code on my computer.

## Hints
- You may want to join the population data _after_ summarizing the affordable housing data (i.e., join populations to sums of units). However, I could also see an approach where you join at the beginning, then aggregate the population column with a method called `first`



In [ ]:
import os
import pandas as pd
from google.colab import drive

def mount_google_drive():
    drive.mount('/content/drive')

def change_working_directory(wd_path):
    os.chdir(wd_path)
    print(f'Current working directory: {os.getcwd()}')

def load_data(housing_csv, demogs_csv):
    housing_projects = pd.read_csv(housing_csv)
    ward_demogs = pd.read_csv(demogs_csv)
    return housing_projects, ward_demogs

def preprocess_data(housing_projects, ward_demogs):
    housing_projects_with_pops = pd.merge(
        housing_projects,
        ward_demogs[['NAME', 'POP100', 'HU100']],
        left_on='MAR_WARD',
        right_on='NAME'
    )

    filtered_df = housing_projects_with_pops[
        (housing_projects_with_pops['STATUS_PUBLIC'] == 'Under Construction') |
        (housing_projects_with_pops['STATUS_PUBLIC'] == 'Pipeline')
    ]

    return filtered_df

def aggregate_and_calculate(df):
    df['total households with incomes up to 60% AMI'] = (
        df['AFFORDABLE_UNITS_AT_0_30_AMI'] +
        df['AFFORDABLE_UNITS_AT_31_50_AMI'] +
        df['AFFORDABLE_UNITS_AT_51_60_AMI']
    )

    grouped_df = df.groupby('MAR_WARD').agg({
        'total households with incomes up to 60% AMI': 'sum',
        'POP100': 'sum',
        'HU100': 'sum'
    }).reset_index()

    grouped_df['Units_Per_Population'] = grouped_df['total households with incomes up to 60% AMI'] / grouped_df['POP100']

    return grouped_df

def identify_wards_by_units(grouped_df):
    average_units_per_population = grouped_df['Units_Per_Population'].mean()

    wards_high_units = grouped_df[grouped_df['Units_Per_Population'] > average_units_per_population]
    wards_low_units = grouped_df[grouped_df['Units_Per_Population'] < average_units_per_population]

    return wards_high_units, wards_low_units

def main():
    wd_path = '/content/drive/MyDrive/ursp688y_shared_data'
    mount_google_drive()
    change_working_directory(wd_path)

    housing_csv = '/content/drive/MyDrive/Tayo Exercise/affordable_housing.csv'
    demogs_csv = 'wards_from_2022.csv'

    housing_projects, ward_demogs = load_data(housing_csv, demogs_csv)
    filtered_df = preprocess_data(housing_projects, ward_demogs)
    grouped_df = aggregate_and_calculate(filtered_df)
    wards_high_units, wards_low_units = identify_wards_by_units(grouped_df)

    print("Wards with high units:", wards_high_units)
    print("Wards with low units:", wards_low_units)

# Execute the main function
if __name__ == '__main__':
    main()


# data_processing_module.py

import os
import pandas as pd

def mount_google_drive():
    from google.colab import drive
    drive.mount('/content/drive')

def change_working_directory(wd_path):
    os.chdir(wd_path)
    print(f'Current working directory: {os.getcwd()}')

def load_data(housing_csv, demogs_csv):
    housing_projects = pd.read_csv(housing_csv)
    ward_demogs = pd.read_csv(demogs_csv)
    return housing_projects, ward_demogs

def preprocess_data(housing_projects, ward_demogs):
    housing_projects_with_pops = pd.merge(
        housing_projects,
        ward_demogs[['NAME', 'POP100', 'HU100']],
        left_on='MAR_WARD',
        right_on='NAME'
    )

    filtered_df = housing_projects_with_pops[
        (housing_projects_with_pops['STATUS_PUBLIC'] == 'Under Construction') |
        (housing_projects_with_pops['STATUS_PUBLIC'] == 'Pipeline')
    ]

    return filtered_df

def aggregate_and_calculate(df):
    df['total households with incomes up to 60% AMI'] = (
        df['AFFORDABLE_UNITS_AT_0_30_AMI'] +
        df['AFFORDABLE_UNITS_AT_31_50_AMI'] +
        df['AFFORDABLE_UNITS_AT_51_60_AMI']
    )
    
    grouped_df = df.groupby('MAR_WARD').agg({
        'total households with incomes up to 60% AMI': 'sum',
        'POP100': 'sum',
        'HU100': 'sum'
    }).reset_index()

    grouped_df['Units_Per_Population'] = grouped_df['total households with incomes up to 60% AMI'] / grouped_df['POP100']
    
    return grouped_df

def identify_wards_by_units(grouped_df):
    average_units_per_population = grouped_df['Units_Per_Population'].mean()

    wards_high_units = grouped_df[grouped_df['Units_Per_Population'] > average_units_per_population]
    wards_low_units = grouped_df[grouped_df['Units_Per_Population'] < average_units_per_population]
    
    return wards_high_units, wards_low_units


In [2]:
# Import your module
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

# Then your code for checking the file exists
abs_path = '//content/drive/MyDrive/Tayo Exercise/affordable_housing.csv'
os.path.isfile(abs_path)


True

In [4]:
wd_path = '/content/drive/MyDrive/ursp688y_shared_data'
os.chdir(wd_path)

print(f'cwd: {os.getcwd()}')

os.path.isfile('affordable_housing.csv')

cwd: /content/drive/.shortcut-targets-by-id/1m3vxo9-OfgjN1ZSn2umVu3lB4xG0HKiQ/ursp688y_shared_data


True

In [5]:
os.listdir()

['affordable_housing.csv',
 '.ipynb_checkpoints',
 '__pycache__',
 'affordable_housing_with_ward_pops (1).csv',
 'affordable_housing_with_ward_pops_test.csv',
 'demo04.py',
 'affordable_housing_calcs_sanford_v2.py',
 'wards_from_2022.csv',
 'data_processing_module.py',
 'wards_from_2022.gsheet',
 'affordable_housing_with_ward_pops.csv',
 'curless_affordable_housing_with_ward.csv',
 'affordable_housing_with_ward_pops_Bardsley3.csv',
 'affordable_housing.gsheet',
 'plot.eps',
 'plot.png']

In [6]:
import pandas as pd

In [7]:
housing_projects = pd.read_csv('/content/drive/MyDrive/Tayo Exercise/affordable_housing.csv')
housing_projects.head()

,X,Y,OBJECTID,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,...,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,MAR_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM
0,-77.009383,38.910255,89281,Ward 6,"1520 North Capitol Street Northwest, Washingto...",Cycle House,Under Construction,DMPED DHCD,18,38.910248,...,4,12,0,0,NaN,331764,399186.36,138042.91,1520 NORTH CAPITOL STREET NW,2024/02/05 05:00:27+00
1,-77.009436,38.906403,89282,Ward 6,"1200 North Capitol Street Northwest, Washingto...",Tyler House Apartments,Completed 2015 to Date,DCHFA,284,38.906396,...,0,284,0,0,NaN,237128,399181.75,137615.28,1200 NORTH CAPITOL STREET NW,2024/02/05 05:00:27+00
2,-77.030061,38.962519,89283,Ward 4,"5922 13th Street Northwest, Washington, Distri...",Valencia Apartments,Completed 2015 to Date,DHCD,29,38.962511,...,0,29,0,0,NaN,243483,397394.87,143845.04,5922 13TH STREET NW,2024/02/05 05:00:27+00
3,-76.950868,38.922332,89284,Ward 5,"3814 Fort Lincoln Drive Northeast, Washington,...",Villages at Dakota Crossing Phase III,Completed 2015 to Date,DMPED,24,38.922333,...,0,0,24,0,NaN,310077,404260.75,139384.60,3814 FORT LINCOLN DRIVE NE,2024/02/05 05:00:27+00
4,-77.033056,38.967357,89285,Ward 4,"1388 Tuckerman Street Northwest, Washington, D...",Vizcaya Apartments,Completed 2015 to Date,DHCD,17,38.967349,...,0,17,0,0,NaN,257527,397135.52,144382.12,1388 TUCKERMAN STREET NW,2024/02/05 05:00:27+00


In [8]:
ward_demogs = pd.read_csv('wards_from_2022.csv')
ward_demogs.head(2)

,WARD,NAME,REP_NAME,WEB_URL,REP_PHONE,REP_EMAIL,REP_OFFICE,WARD_ID,LABEL,STUSAB,...,P0050009,P0050010,OBJECTID,GLOBALID,CREATED_USER,CREATED_DATE,LAST_EDITED_USER,LAST_EDITED_DATE,SHAPEAREA,SHAPELEN
0,8,Ward 8,"Trayon White, Sr.",https://www.dccouncil.us/council/councilmember...,(202) 724-8045,twhite@dccouncil.us,"1350 Pennsylvania Ave, Suite 400, NW 20004",8,Ward 8,DC,...,563,1745,1,{E31550AE-6FAE-4B74-909F-52B283BFAF68},NaN,NaN,NaN,NaN,0,0
1,6,Ward 6,Charles Allen,https://www.dccouncil.us/council/councilmember...,(202) 724-8072,callen@dccouncil.us,"1350 Pennsylvania Ave, Suite 110, NW 20004",6,Ward 6,DC,...,255,887,2,{765C4F49-9292-4BDB-AA24-39F4EE43359F},NaN,NaN,JLAY,2023/12/07 20:08:04+00,0,0


In [9]:
housing_projects.head(2)

,X,Y,OBJECTID,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,...,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,AFFORDABLE_UNITS_AT_61_80_AMI,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,MAR_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM
0,-77.009383,38.910255,89281,Ward 6,"1520 North Capitol Street Northwest, Washingto...",Cycle House,Under Construction,DMPED DHCD,18,38.910248,...,4,12,0,0,NaN,331764,399186.36,138042.91,1520 NORTH CAPITOL STREET NW,2024/02/05 05:00:27+00
1,-77.009436,38.906403,89282,Ward 6,"1200 North Capitol Street Northwest, Washingto...",Tyler House Apartments,Completed 2015 to Date,DCHFA,284,38.906396,...,0,284,0,0,NaN,237128,399181.75,137615.28,1200 NORTH CAPITOL STREET NW,2024/02/05 05:00:27+00


In [10]:
for name in ward_demogs['NAME'].sort_values().unique():
    if name in housing_projects['MAR_WARD'].unique():
        print(f'{name} in both')
    else:
        print(f'{name} not in housing projects')

Ward 1 in both
Ward 2 in both
Ward 3 in both
Ward 4 in both
Ward 5 in both
Ward 6 in both
Ward 7 in both
Ward 8 in both


In [11]:
housing_projects_with_pops = pd.merge(housing_projects, ward_demogs, left_on='MAR_WARD', right_on='NAME')

# housing_projects_with_pops.columns.tolist()

In [12]:
housing_projects_with_pops = pd.merge(
    housing_projects,
    ward_demogs[['NAME','POP100','HU100']],
    left_on='MAR_WARD',
    right_on='NAME')

housing_projects_with_pops.head(2)

,X,Y,OBJECTID,MAR_WARD,ADDRESS,PROJECT_NAME,STATUS_PUBLIC,AGENCY_CALCULATED,TOTAL_AFFORDABLE_UNITS,LATITUDE,...,AFFORDABLE_UNITS_AT_81_AMI,CASE_ID,MAR_ID,XCOORD,YCOORD,FULLADDRESS,GIS_LAST_MOD_DTTM,NAME,POP100,HU100
0,-77.009383,38.910255,89281,Ward 6,"1520 North Capitol Street Northwest, Washingto...",Cycle House,Under Construction,DMPED DHCD,18,38.910248,...,0,NaN,331764,399186.36,138042.91,1520 NORTH CAPITOL STREET NW,2024/02/05 05:00:27+00,Ward 6,84266,52768
1,-77.009436,38.906403,89282,Ward 6,"1200 North Capitol Street Northwest, Washingto...",Tyler House Apartments,Completed 2015 to Date,DCHFA,284,38.906396,...,0,NaN,237128,399181.75,137615.28,1200 NORTH CAPITOL STREET NW,2024/02/05 05:00:27+00,Ward 6,84266,52768


In [13]:
# Filtering the DataFrame for projects that are either under construction or in the pipeline
filtered_df = housing_projects_with_pops[
    (housing_projects_with_pops['STATUS_PUBLIC'] == 'Under Construction') |
    (housing_projects_with_pops['STATUS_PUBLIC'] == 'Pipeline')
]

# Selecting the specified columns
result_df = filtered_df[[
    'MAR_WARD', 'STATUS_PUBLIC', 'AFFORDABLE_UNITS_AT_0_30_AMI',
    'AFFORDABLE_UNITS_AT_31_50_AMI', 'AFFORDABLE_UNITS_AT_51_60_AMI',
    'POP100', 'HU100'
]]

result_df


,MAR_WARD,STATUS_PUBLIC,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,POP100,HU100
0,Ward 6,Under Construction,2,4,12,84266,52768
9,Ward 6,Pipeline,140,0,139,84266,52768
31,Ward 6,Pipeline,67,48,0,84266,52768
46,Ward 6,Pipeline,0,0,0,84266,52768
48,Ward 6,Pipeline,69,0,0,84266,52768
...,...,...,...,...,...,...,...
869,Ward 3,Under Construction,52,41,0,85301,44109
870,Ward 3,Under Construction,0,4,37,85301,44109
871,Ward 3,Pipeline,2,26,27,85301,44109
872,Ward 3,Pipeline,0,0,23,85301,44109


In [14]:
# Grouping the DataFrame by 'MAR_WARD' and summing up the columns for each group
grouped_sum_df = result_df.groupby('MAR_WARD').sum()

grouped_sum_df


<ipython-input-14-6efd8545621f>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_sum_df = result_df.groupby('MAR_WARD').sum()


,AFFORDABLE_UNITS_AT_0_30_AMI,AFFORDABLE_UNITS_AT_31_50_AMI,AFFORDABLE_UNITS_AT_51_60_AMI,POP100,HU100
MAR_WARD,,,,,
Ward 1,258,411,785,3070260,1644984
Ward 2,23,17,220,1342275,798255
Ward 3,77,87,280,1620719,838071
Ward 4,126,229,277,4910280,2009700
Ward 5,600,683,969,7258977,3385314
Ward 6,659,575,1200,4044768,2532864
Ward 7,528,1140,642,3170345,1441816
Ward 8,1189,1760,1343,4859022,2232348


In [15]:
# Summing the specified columns for each row to create a new column
result_df['total households with incomes up to 60% AMI'] = (
    result_df['AFFORDABLE_UNITS_AT_0_30_AMI'] +
    result_df['AFFORDABLE_UNITS_AT_31_50_AMI'] +
    result_df['AFFORDABLE_UNITS_AT_51_60_AMI']
)

grouped_sum_df = result_df.groupby('MAR_WARD').agg({
    'total households with incomes up to 60% AMI': 'sum',
    # Adding other columns here
    'POP100': 'sum',
    'HU100': 'sum'
}).reset_index()

grouped_sum_df


<ipython-input-15-692384a8772d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['total households with incomes up to 60% AMI'] = (


,MAR_WARD,total households with incomes up to 60% AMI,POP100,HU100
0,Ward 1,1454,3070260,1644984
1,Ward 2,260,1342275,798255
2,Ward 3,444,1620719,838071
3,Ward 4,632,4910280,2009700
4,Ward 5,2252,7258977,3385314
5,Ward 6,2434,4044768,2532864
6,Ward 7,2310,3170345,1441816
7,Ward 8,4292,4859022,2232348


In [16]:
# Assuming 'grouped_sum_df' contains the aggregated data you're interested in,
# Calculate the ratio of total households with incomes up to 60% AMI per 'POP100' and add it as a new column
grouped_sum_df['Units_Per_Population'] = grouped_sum_df['total households with incomes up to 60% AMI'] / grouped_sum_df['POP100']

# If you intend to create a new DataFrame 'Housing_Pop' based on 'grouped_sum_df' with the new calculation
Housing_Pop = grouped_sum_df.copy()
Housing_Pop


,MAR_WARD,total households with incomes up to 60% AMI,POP100,HU100,Units_Per_Population
0,Ward 1,1454,3070260,1644984,0.000474
1,Ward 2,260,1342275,798255,0.000194
2,Ward 3,444,1620719,838071,0.000274
3,Ward 4,632,4910280,2009700,0.000129
4,Ward 5,2252,7258977,3385314,0.000310
5,Ward 6,2434,4044768,2532864,0.000602
6,Ward 7,2310,3170345,1441816,0.000729
7,Ward 8,4292,4859022,2232348,0.000883


In [17]:
# Calculate the ratio of total households with incomes up to 60% AMI per 100 people
grouped_sum_df['Units_Per_Population'] = grouped_sum_df['total households with incomes up to 60% AMI'] / grouped_sum_df['POP100']

# Calculate the average ratio for comparison
average_units_per_population = grouped_sum_df['Units_Per_Population'].mean()

# Identify wards with disproportionately high numbers of housing units
wards_high_units = grouped_sum_df[grouped_sum_df['Units_Per_Population'] > average_units_per_population]

# Identify wards with disproportionately low numbers of housing units
wards_low_units = grouped_sum_df[grouped_sum_df['Units_Per_Population'] < average_units_per_population]
wards_high_units
wards_low_units


,MAR_WARD,total households with incomes up to 60% AMI,POP100,HU100,Units_Per_Population
1,Ward 2,260,1342275,798255,0.000194
2,Ward 3,444,1620719,838071,0.000274
3,Ward 4,632,4910280,2009700,0.000129
4,Ward 5,2252,7258977,3385314,0.000310


In [18]:
wards_high_units

,MAR_WARD,total households with incomes up to 60% AMI,POP100,HU100,Units_Per_Population
0,Ward 1,1454,3070260,1644984,0.000474
5,Ward 6,2434,4044768,2532864,0.000602
6,Ward 7,2310,3170345,1441816,0.000729
7,Ward 8,4292,4859022,2232348,0.000883
